# Latent Factor Model - Recommender System

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

In [ ]:
np.set_printoptions(threshold = 100)

## Preprocessing

In [ ]:
def preprocess():
    
    '''
        Constructs ratings matrix from the Movie Lens Database file having information on users and their ratings.
        
        Then, extracts all non-zero data entries (userID, movieID, rating) from the sparse matrix, which then forms
        the dataset for use in the Latent Factor Model.
        
        Further, splits the dataset of known ratings into train, test and validation datasets in the ratio 0.75:0.15:0.1.
        
        Returns: 
            1) Constructed sparse ratings matrix
            2) Samples for training
            3) Samples for testing
            4) Samples for validation

    '''
    
    # ratingsInfo = pd.read_csv('ratingsActual.csv', usecols = [0,1,2], header = None)  
    
    # print(len(ratingsInfo))    # 1000209
    
    
    
    ######## Constructing Ratings Matrix ###########
    
    
    # ratingsMatrix = np.zeros((6040,3952))
    # ratingsCount = 0
    # for i in range(len(ratingsInfo)):
    #    ratingsMatrix[ratingsInfo.iloc[i,0]-1][ratingsInfo.iloc[i,1]-1] = ratingsInfo.iloc[i,2]
    #    ratingsCount += 1
        
    # np.save("ratingsMatrix.npy",ratingsMatrix)    
    
    ratingsMatrix = np.load("ratingsMatrix.npy")
    totalUsers = ratingsMatrix.shape[0]
    totalMovies = ratingsMatrix.shape[1]
    
    ###################################################
    
    
    
    
    ######## Getting all non-zero samples from Ratings Matrix for use in the Latent Factor Model ###########
    
    
    totalSamples = [(user,movie,ratingsMatrix[user][movie]) for user in range(totalUsers) for movie in range(totalMovies) if ratingsMatrix[user][movie]>0] 
    
    ratingsCount = len(totalSamples)
    
    # print(len(totalSamples))    # 1000209

    np.random.shuffle(totalSamples)    
    
    ########################################################################################################
    
    
    
    
    
    ######## Splitting Known Ratings into Train, Test and Validation Sets:   (In ratio *0.75 : 0.15 : 0.10* respectively) ########
    

    trainSamples = totalSamples[ : int(0.75 * ratingsCount)]
    testSamples = totalSamples[int(0.75 * ratingsCount) : int(0.90 * ratingsCount)]
    valSamples = totalSamples[int(0.90 * ratingsCount) : ]
    
    ###########################################################################################################
    
    
    
    return ratingsMatrix, trainSamples, testSamples, valSamples

## Stochastic Gradient Descent Algorithm

### Initializing the matrices P, Q, and biases b, b_u, b_m

In [ ]:
def initialize_weights (trainSamples, K, totalUsers, totalMovies):
    '''
        Initializes the user-feature and item-feature matrices with values from a normal distribution,
        scaled by 1/K.
        The global bias b is the mean of all the known ratings in the matrix.
        The user and item biases are initialized to zeros.
    '''

    # print(totalUsers, totalMovies)
    
    P = np.random.normal(scale = 1/K, size = (totalUsers, K))
    Q = np.random.normal(scale = 1/K, size = (totalMovies, K))

    b_u = np.zeros(totalUsers)
    b_m = np.zeros(totalMovies)
    
    ratings = np.array([rating for (user,movie,rating) in trainSamples])
    b = np.mean(ratings)
    
    # print(b) ---- > b = 3.581564453029317 for full rating matrix
    
    return P, Q, b_u, b_m, b


### Predicting ratings

In [ ]:
def get_rating (i, j, b_u_i, b_m_j, b, P_i, Q_j):
    '''
        Returns a single value for the predicted rating for a user i and item j.
    '''
    
    rating = b + b_u_i + b_m_j + np.dot(P_i, Q_j.T)
    
    return rating

In [ ]:
def predict_all_ratings (P, Q, b_u, b_m, b):
    '''
        Based on the optimized user-feature matrix, item-feature matrix and biases (user, item, global),
        this function returns a full user-item matrix containing the predicted values for each user-item pair
        in the original ratings matrix.
    '''
    
    allRatings = b + b_u[:,np.newaxis] + b_m[np.newaxis,:] + np.matmul(P,Q.T)   
    allRatings [allRatings > 5] = 5
    allRatings [allRatings < 1] = 1
    
    return allRatings

### Evaluating the model

In [ ]:
def get_errors (predictedRatingMatrix, actualRatings) :
    '''
        Given the predicted rating matrix, and actual ratings, this function returns the following
        errors:
        1) Sum of Squares Error (SSE)
        2) Mean Square Error (MSE)
        3) Root Mean Square Error (RMSE)
        4) Mean Absolute Error (MAE)
    '''
    
    SSE = 0
    SAE = 0
    
    for user, movie, trueRating in actualRatings :
        
        SSE += (trueRating - predictedRatingMatrix[user,movie]) ** 2
        SAE += abs(trueRating - predictedRatingMatrix[user,movie])
    
    MSE = SSE / len(actualRatings)
    RMSE = MSE ** 0.5
    MAE = SAE / len(actualRatings)
    
    return SSE, MSE, RMSE, MAE

### Stochastic Gradient Descent - Using Alternate Least Squares Technique

In [ ]:
def stochastic_gradient_descent_ALS (alpha, epochs, trainSamples, K, ratingsMatrix, lambda_reg = 0):
    '''
    
        Performs stochastic gradient descent on the training samples, and returns the optimized user-feature
        and item-feature matrices P and Q respectively, along with the optimized biases for users and items, 
        and the global bias.
        
        This variation uses the alternate least squares (ALS) method in order to guarantee convergence.
        
        In ALS, in each epoch, using stochastic gradient descent, we find the optimal P matrix without changing Q.
        Consequently, using the optimal P matrix hence obtained, we perform stochastic gradient descent for matrix Q.
        This procedure repeats for each epoch, to yield the final optimal matrices P (user-feature), and Q (item-feature).
        
    '''
    
    totalUsers = ratingsMatrix.shape[0]
    totalMovies = ratingsMatrix.shape[1]
    
    P, Q, b_u, b_m, b = initialize_weights(trainSamples, K, totalUsers, totalMovies)
    
    cost = []
    
    for epoch in tqdm_notebook(range(epochs)): 
        
        for i in tqdm_notebook(range(len(trainSamples))):
            
            user, movie, ratingActual = trainSamples[i]
            
            ratingPredicted = get_rating(user, movie, b_u[user], b_m[movie], b, P[user,:], Q[movie,:])
            error = ratingActual - ratingPredicted
            
            b_u[user] += alpha * (2 * error - lambda_reg * b_u[user])
            b_m[movie] += alpha * (2 * error - lambda_reg * b_m[movie])
            
            P[user,:] += alpha * (2 * error * Q[movie,:] - lambda_reg * P[user,:])
            
        for i in tqdm_notebook(range(len(trainSamples))):
            
            user, movie, ratingActual = trainSamples[i]
            
            ratingPredicted = get_rating(user, movie, b_u[user], b_m[movie], b, P[user,:], Q[movie,:])
            error = ratingActual - ratingPredicted
            
            b_u[user] += alpha * (2 * error - lambda_reg * b_u[user])
            b_m[movie] += alpha * (2 * error - lambda_reg * b_m[movie])
            
            Q[movie,:] += alpha * (2 * error * P[user,:] - lambda_reg * Q[movie,:])
            
        _,_,RMSE,MAE = get_errors( predict_all_ratings(P, Q, b_u, b_m, b), trainSamples)

        cost.append((RMSE,MAE))
    
    return P, Q, b_u, b_m, b, cost

### Standard Stochastic Gradient Descent

In [ ]:
def stochastic_gradient_descent (alpha, epochs, trainSamples, K, ratingsMatrix, lambda_reg = 0):
    '''
        Performs stochastic gradient descent on the training samples, and returns the optimized user-feature
        and item-feature matrices P and Q respectively, along with the optimized biases for users and items, 
        and the global bias.
    '''
    
    totalUsers = ratingsMatrix.shape[0]
    totalMovies = ratingsMatrix.shape[1]
    
    P, Q, b_u, b_m, b = initialize_weights(trainSamples, K, totalUsers, totalMovies)
    
    cost = []
    
    for epoch in tqdm_notebook(range(epochs)): 
        
        for i in tqdm_notebook(range(len(trainSamples))):
            
            user, movie, ratingActual = trainSamples[i]
            
            ratingPredicted = get_rating(user, movie, b_u[user], b_m[movie], b, P[user,:], Q[movie,:])
            error = ratingActual - ratingPredicted

            b_u[user] += alpha * (2 * error - lambda_reg * b_u[user])
            b_m[movie] += alpha * (2 * error - lambda_reg * b_m[movie])
            
            P_i = P[user,:]
            P[user,:] += alpha * (2 * error * Q[movie,:] - lambda_reg * P[user,:])
            Q[movie,:] += alpha * (2 * error * P_i - lambda_reg * Q[movie,:])
            
        _,_,RMSE,MAE = get_errors( predict_all_ratings(P, Q, b_u, b_m, b), trainSamples)

        cost.append((RMSE,MAE))
    
    return P, Q, b_u, b_m, b, cost

### Hyperparameter Tuning : (Regularization Parameter *lambda* and Number of Factors *K*)

In [ ]:
def find_best_lambda (alpha, epochs, trainSamples, valSamples, K, ratingsMatrix, l, r):
    '''
        Finds the best regularization hyperparameter for the model, using Binary Search.
        
        The training using each candidate lambda value is done using the Training Samples,
        and the evaluation of the performance of each such model is done using the Validation Samples.
        
        Based on the RMSE score on the Validation Set, the hyperparameter is adjusted so as to converge
        to the minimum RMSE for the Validation Set. 
        
        The model can then be evaluated on the Test Set.
        
    '''
    lambdas = []
    
    for i in tqdm_notebook(range(5)):
        
        lambda_ = (l + r) / 2
        
        lambda_right = lambda_ + 0.01*lambda_
        lambda_left = lambda_ - 0.01*lambda_
        
        P, Q, b_u, b_m, b,_ = stochastic_gradient_descent_ALS(alpha, epochs, trainSamples, K, ratingsMatrix, lambda_)
        Pr, Qr, b_ur, b_mr, br,_ = stochastic_gradient_descent_ALS(alpha, epochs, trainSamples, K, ratingsMatrix, lambda_right)
        Pl, Ql, b_ul, b_ml, bl,_ = stochastic_gradient_descent_ALS(alpha, epochs, trainSamples, K, ratingsMatrix, lambda_left)

        _,_,RMSE,MAE = get_errors (predict_all_ratings(P, Q, b_u, b_m, b), valSamples)
        _,_,RMSE_right,MAE_right = get_errors (predict_all_ratings(Pr, Qr, b_ur, b_mr, br), valSamples)
        _,_,RMSE_left,MAE_left = get_errors (predict_all_ratings(Pl, Ql, b_ul, b_ml, bl), valSamples)
        
        
        print("L_left = {}, L = {}, L_right = {}".format(lambda_left, lambda_, lambda_right))
        print("RMSE_left = {}, RMSE = {}, RMSE_right = {}".format(RMSE_left, RMSE, RMSE_right))
        lambdas.append(lambda_)
        
        if RMSE_right > RMSE and RMSE_left > RMSE:
            
            break
            
        elif RMSE_left < RMSE:
            
            r = lambda_
            
        else:
            
            l = lambda_
        
    return lambda_, lambdas, RMSE, MAE, P, Q, b_u, b_m, b

In [ ]:
def tune_hyperparameters (possibleK, alpha, epochs, trainSamples, testSamples, valSamples, ratingsMatrix, l, r) :
    '''
        Tunes the hyperparamters K and lambda, by finding the best lambda for each possible K value,using
        Binary Search.
        The best combination of K and lambda, giving the least RMSE over the validation set can be chosen as the 
        optimal configuration for the model, and its performance evaluated on the test set.
    '''
    
    lambdasK = []
    bestLambdaK = []
    performanceOnVal = []
    performanceOnTest = []
    
    for K in possibleK :
        
        print("\nK = ",K)
        print("===========================================================")
        
        P, Q, b_u, b_m, b,_ = stochastic_gradient_descent_ALS(alpha, epochs, trainSamples, K, ratingsMatrix)
        _,_,RMSE,MAE = get_errors (predict_all_ratings(P, Q, b_u, b_m, b), valSamples)
        
        print("Without Regularization, (on Validation Set):")
        print("RMSE = {}, MAE = {} \n".format(RMSE, MAE))
        print("Finding best regularization hyperparameter ...")
        
        lambda_, lambdas, RMSE, MAE,P, Q, b_u, b_m, b = find_best_lambda(alpha, epochs, trainSamples, valSamples, K, ratingsMatrix, l, r)
        
        bestLambdaK.append(lambda_)
        lambdasK.append(lambdas)
        
        print("After Regularization, (on Validation Set):")
        print("RMSE = {}, MAE = {} \n".format(RMSE, MAE))
        print('Best lambda = ', lambda_)
        performanceOnVal.append((RMSE,MAE))
        
        _,_,RMSE, MAE = get_errors (predict_all_ratings(P, Q, b_u, b_m, b), testSamples)
        
        print('\nPerformance on Test Set:')
        print("RMSE = {}, MAE = {} \n".format(RMSE, MAE))
        
        performanceOnTest.append((RMSE,MAE))
        
    return lambdasK, bestLambdaK, performanceOnVal, performanceOnTest

In [ ]:
def find_best_K (possibleK, alpha, epochs, trainSamples, testSamples, ratingsMatrix, lambda_reg = 0):
    
    '''
        For a chosen lambda value, performs stochastic gradient descent for each candidate K value,
        and returns the resulting performance of each K value for the given regularization parameter, on 
        the test set.
    '''

    testRMSE = []

    for K in possibleK:
        
        print("K= ",K)
        
        P, Q, b_u, b_m, b, cost = stochastic_gradient_descent_ALS(alpha, epochs, trainSamples, K, ratingsMatrix, lambda_reg)
        _,_,RMSE, MAE = get_errors (predict_all_ratings(P, Q, b_u, b_m, b), testSamples)
        testRMSE.append((RMSE,MAE))
    
    return testRMSE

# Testing the Algorithm

In [ ]:
if __name__ == '__main__':
    
    # P, Q, b_u, b_m, b, cost500 = stochastic_gradient_descent_ALS(0.01, 20, trainSamples, 500, ratingsMatrix,0.01)
    # P = np.load("P500.npy")
    # Q = np.load("Q500.npy")
    # b_m = np.load("b_m500.npy")
    # b_u = np.load("b_u500.npy")
    # b = np.load("b500.npy")
    
    ratingsMatrix, trainSamples, testSamples, valSamples = preprocess()
    
    P, Q, b_u, b_m, b, cost = stochastic_gradient_descent_ALS(0.01, 20, trainSamples, 1000, ratingsMatrix, 0.01)
    
    allRatings = predict_all_ratings (P, Q, b_u, b_m, b)

    _,_,RMSE,MAE = get_errors(allRatings, testSamples)

    print("RMSE= ",RMSE)
    print("MAE= ",MAE)

# Results

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 5

 Test RMSE = 0.8793194552497463
 
 Test MAE = 0.6877353757628977

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 10

 Test RMSE = 0.8905157855499776
 
 Test MAE = 0.6929465135259187

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 25

 Test RMSE = 0.9391536000818118
 
 Test MAE = 0.7275229466603019

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 50

 Test RMSE = 0.9930651990946064
 
 Test MAE = 0.7699266990632744

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 100

 Test RMSE = 1.0102510579920232
 
 Test MAE = 0.7875524567174196

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 150

 Test RMSE = 0.9870828616500531
 
 Test MAE = 0.7722730412958273

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 200

 Test RMSE = 0.9568730897273094
 
 Test MAE = 0.7490492325871345

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K = 500

 Test RMSE = 0.9039870700310948
 
 Test MAE = 0.7045523325010874

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0.01, K=1000

 Train RMSE = 0.14170220361792696

 Test RMSE = 0.8481153815446396
 
 Test MAE = 0.6665757266480518

### SGD - ALS : 20 epochs, alpha = 0.01, lambda = 0, K=2000

 Train RMSE = 0.10471021346343064

 Test RMSE = 0.9109825227707479
 
 Test MAE = 0.7132322662513719